# Participant characteristics

In [37]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel
# import timedelta
from datetime import timedelta

In [38]:
df_cut_final = pd.read_csv('../../data/tidy/paired_data/final_data_for_analysis_iob_check.csv')

In [39]:
# times glc_change, glc, start_roc,  'Average glucose (mmol/L)', 'SD (mmol/L)', 'AUC (mmol h/L)', 'MAGE (mmol/L)' by 18

df_cut_final['glc_change'] = df_cut_final['glc_change']*18
df_cut_final['glc'] = df_cut_final['glc']*18
df_cut_final['start_roc'] = df_cut_final['start_roc']*18


In [40]:
df_weight = pd.read_csv('../../../hypo-predict/data/tidy_data/demographics_df.csv')
df_weight.drop_duplicates(subset=['ID'], keep='first', inplace=True)
df_cut_final = df_cut_final.merge(df_weight[['ID', 'weight']], on='ID', how='inner')
df_cut_final['iob_kg'] = df_cut_final['IOB'] / df_cut_final['weight']

In [41]:
all_metrics = pd.read_csv('../../../hypo-predict/results/metrics_glycemic_control.csv')

In [42]:
df_cut_final['start_datetime'] = pd.to_datetime(df_cut_final['start_datetime'])

In [43]:
# Categorize time_of_day based on the time column
df_cut_final['time_of_day'] = df_cut_final['start_datetime'].apply(
    lambda x: 'morning' if 6 <= x.hour < 12 else
              'afternoon' if 12 <= x.hour < 18 else
              'evening'
)

In [44]:
df_cut_final.shape[0]/2

1546.0

In [45]:
df_cut_final[df_cut_final['ID'].str.startswith('dexip')].shape[0]/2

253.0

In [46]:
df_cut_final.ID

0          helm_1
1          helm_1
2          helm_1
3          helm_4
4          helm_4
          ...    
3087    dexip_531
3088    dexip_535
3089    dexip_535
3090    dexip_540
3091    dexip_540
Name: ID, Length: 3092, dtype: object

## Demographics & glycemic control metrics

In [47]:
ov_cat_vars = ['insulin_modality', 'race', 'sex']
ov_num_vars = ['age', 'bmi', 'hba1c', 'years_since_diagnosis', 'Average glucose (mmol/L)',
 'eA1c (%)',
 'SD (mmol/L)',
 'CV (%)',
 'AUC (mmol h/L)',
 'LBGI',
 'HBGI',
 'MAGE (mmol/L)',
 'TIR normal (%)',
 'tir_hypo',
 'tir_hyper'
 ]

In [48]:
df_demo = df_cut_final.drop_duplicates(subset=['ID'], keep='first')

In [49]:
df_demo_glyc = df_demo.merge(all_metrics, on='ID', how='left')

In [50]:
df_demo_glyc.shape

(482, 88)

In [51]:
df_demo_glyc['tir_hypo'] = df_demo_glyc['TIR level 1 hypoglycemia (%)'] + df_demo_glyc['TIR level 2 hypoglycemia (%)']
df_demo_glyc['tir_hyper'] = df_demo_glyc['TIR level 1 hyperglycemia (%)'] + df_demo_glyc['TIR level 2 hyperglycemia (%)']

In [52]:
df_demo_glyc['Average glucose (mmol/L)'] = df_demo_glyc['Average glucose (mmol/L)']*18
df_demo_glyc['SD (mmol/L)'] = df_demo_glyc['SD (mmol/L)']*18
df_demo_glyc['AUC (mmol h/L)'] = df_demo_glyc['AUC (mmol h/L)']*18
df_demo_glyc['MAGE (mmol/L)'] = df_demo_glyc['MAGE (mmol/L)']*18

In [53]:
import pandas as pd
import numpy as np

# Example dataframe
df = df_demo_glyc  # Replace with your actual dataframe

# Initialize dictionaries to store results
num_summary = {}
cat_summary = {}

# Numerical variables: median [IQR]
for col in ov_num_vars:
    median = np.around(df[col].median(), 3)
    q1 = np.around(df[col].quantile(0.25), 3)
    q3 = np.around(df[col].quantile(0.75), 3)
    num_summary[col] = f"{median} [{q1}, {q3}]"

# Categorical variables: category: count (%)
for col in ov_cat_vars:
    counts = df[col].value_counts()
    percentages = (counts / counts.sum()) * 100
    cat_summary[col] = [
        f"{category}: {count} ({percent:.2f}%)" 
        for category, count, percent in zip(counts.index, counts, percentages)
    ]

# Convert results to DataFrames for better readability
num_summary_df = pd.DataFrame(num_summary, index=['Summary']).T
cat_summary_df = pd.DataFrame.from_dict(cat_summary, orient='index').T

# Print results
print("Numerical Summary:")
print(num_summary_df)

print("\nCategorical Summary:")
print(cat_summary_df)


Numerical Summary:
                                             Summary
age                                26.0 [18.0, 39.0]
bmi                            24.061 [21.8, 26.605]
hba1c                         50.82 [45.355, 56.284]
years_since_diagnosis               12.0 [9.0, 20.0]
Average glucose (mmol/L)   151.063 [138.281, 171.91]
eA1c (%)                        6.907 [6.461, 7.636]
SD (mmol/L)                  51.799 [43.855, 61.943]
CV (%)                       34.121 [30.398, 37.417]
AUC (mmol h/L)            136.563 [125.918, 156.838]
LBGI                            0.619 [0.325, 0.978]
HBGI                            5.558 [3.635, 8.829]
MAGE (mmol/L)             125.704 [106.532, 149.559]
TIR normal (%)                71.88 [57.535, 80.205]
tir_hypo                        2.075 [0.822, 3.672]
tir_hyper                     25.645 [15.715, 39.27]

Categorical Summary:
            insulin_modality                                       race  \
0  closed_loop: 226 (46.89%)        

In [17]:
num_summary_df

,Summary
age,"26.0 [18.0, 39.0]"
bmi,"24.061 [21.8, 26.605]"
hba1c,"50.82 [45.355, 56.284]"
years_since_diagnosis,"12.0 [9.0, 20.0]"
Average glucose (mmol/L),"8.392 [7.682, 9.551]"
eA1c (%),"6.907 [6.461, 7.636]"
SD (mmol/L),"2.878 [2.436, 3.441]"
CV (%),"34.121 [30.398, 37.417]"
AUC (mmol h/L),"7.587 [6.995, 8.713]"
LBGI,"0.619 [0.325, 0.978]"


In [18]:
cat_summary_df

,insulin_modality,race,sex
0,closed_loop: 226 (46.89%),WHITE: 435 (90.25%),female: 329 (68.26%)
1,pump: 183 (37.97%),NOT REPORTED: 14 (2.90%),male: 153 (31.74%)
2,mdi: 73 (15.15%),MULTIPLE: 11 (2.28%),None
3,None,ASIAN: 10 (2.07%),None
4,None,BLACK/AFRICAN AMERICAN: 8 (1.66%),None
5,None,AMERICAN INDIAN/ALASKAN NATIVE: 2 (0.41%),None
6,None,UNKNOWN: 2 (0.41%),None


## Bout characteristics

In [54]:
num_vars = ['glc_change', 'duration', 'iob_kg', 'glc', 'start_roc', 'cv']
cat_vars = ['hypo', 'form_of_exercise', 'time_of_day', 'form_of_exercise', 'intensity']


In [22]:
df.columns

Index(['bout_id', 'ID', 'start_datetime', 'duration', 'intensity',
       'type_of_exercise', 'finish_datetime', 'month', 'day', 'day_of_week',
       'time_of_day', 'form_of_exercise', 'time', 'glc', 'start_roc', 'IOB',
       'iob_kg', 'cv', 'glc_post_20', 'hypo', 'exercise', 'glc_change', 'aer',
       'ana', 'mix', 'age', 'sex', 'race', 'hba1c', 'bmi',
       'years_since_diagnosis', 'insulin_modality', 'hour_of_day', 'light',
       'moderate', 'vigorous', 'female', 'iob_bin', 'age_bin',
       'years_since_diagnosis_bin', 'bmi_bin', 'hba1c_bin', 'start_roc_bin',
       'glc_bin', 'pump', 'mdi', 'closed_loop', 'iob_quadratic',
       'start_roc_quadratic', 'cv_quadratic', 'glc_quadratic', 'age_quadratic',
       'bmi_quadratic', 'iob_cubed', 'start_roc_cubed', 'cv_cubed',
       'glc_cubed', 'age_cubed', 'bmi_cubed', 'weight'],
      dtype='object')

In [20]:
import pandas as pd
import numpy as np

# Example dataframe
df = df_cut_final  # Replace with your actual dataframe

# Initialize dictionaries to store results
num_summary = {}
cat_summary = {}

# Group by 'exercise'
grouped = df.groupby('exercise')

# Numerical variables: median [IQR]
for exercise_group, group_data in grouped:
    num_summary[exercise_group] = {}
    for col in num_vars:
        median = np.around(group_data[col].median(), 3)
        q1 = np.around(group_data[col].quantile(0.25), 3)
        q3 = np.around(group_data[col].quantile(0.75), 3)
        num_summary[exercise_group][col] = f"{median} [{q1}, {q3}]"

# Categorical variables: category: count (%)
for exercise_group, group_data in grouped:
    cat_summary[exercise_group] = {}
    for col in cat_vars:
        counts = group_data[col].value_counts()
        percentages = (counts / counts.sum()) * 100
        cat_summary[exercise_group][col] = [
            f"{category}: {count} ({percent:.2f}%)" 
            for category, count, percent in zip(counts.index, counts, percentages)
        ]

# Convert results to DataFrames for better readability
num_summary_df = pd.DataFrame(num_summary).T  # Transpose for readability
cat_summary_df = {group: pd.DataFrame.from_dict(cat_summary[group], orient='index').T 
                  for group in cat_summary}

# Print results
print("Numerical Summary:")
print(num_summary_df)
overall_num_ov
print("\nCategorical Summary:")
for group, df in cat_summary_df.items():
    print(f"\nExercise Group: {group}")
    print(df)


KeyError: 'avg_glucose'

In [ ]:
cat_summary_df

{0:                     hypo      form_of_exercise              time_of_day  \
 0  False: 1837 (100.00%)  none: 1837 (100.00%)    evening: 755 (41.10%)   
 1                   None                  None  afternoon: 627 (34.13%)   
 2                   None                  None    morning: 455 (24.77%)   
 
               intensity  
 0  -1.0: 1837 (100.00%)  
 1                  None  
 2                  None  ,
 1:                     hypo    form_of_exercise              time_of_day  \
 0  False: 1837 (100.00%)  aer: 1050 (57.16%)  afternoon: 829 (45.13%)   
 1                   None   mix: 637 (34.68%)    evening: 568 (30.92%)   
 2                   None    ana: 150 (8.17%)    morning: 440 (23.95%)   
 
            intensity  
 0  0.0: 914 (49.76%)  
 1  1.0: 772 (42.03%)  
 2   2.0: 151 (8.22%)  }

In [ ]:
num_summary_df

,glc_change,duration,iob_kg,glc,start_roc,cv,avg_glucose,sd_glucose,time_below_3_9,time_3_9_to_10,time_above_10
0,"-0.34 [-1.33, 0.66]","23.0 [20.0, 30.0]","0.012 [0.003, 0.03]","11.72 [10.78, 13.33]","0.003 [-0.03, 0.047]","6.086 [3.693, 10.388]","11.6 [10.282, 13.289]","0.75 [0.456, 1.197]","0.0 [0.0, 0.0]","0.0 [0.0, 0.417]","1.0 [0.583, 1.0]"
1,"-2.22 [-4.11, -0.39]","23.0 [20.0, 30.0]","0.013 [0.004, 0.03]","11.78 [10.78, 13.39]","0.003 [-0.033, 0.047]","6.176 [3.599, 10.585]","11.569 [10.241, 13.342]","0.76 [0.447, 1.221]","0.0 [0.0, 0.0]","0.0 [0.0, 0.385]","1.0 [0.6, 1.0]"


In [ ]:
df_cut_final['hour'] = df_cut_final['time'].dt.hour

AttributeError: Can only use .dt accessor with datetimelike values

In [ ]:
df_cut_final[df_cut_final['exercise'] == 0]['hour'].value_counts()

Series([], Name: count, dtype: int64)

In [ ]:
def summarize_by_exercise(df, num_vars, cat_vars):
    num_summary = {}
    cat_summary = {}

    grouped = df.groupby('exercise')

    # Numerical variables
    for exercise_group, group_data in grouped:
        num_summary[exercise_group] = {}
        for col in num_vars:
            median = np.around(group_data[col].median(), 3)
            q1 = np.around(group_data[col].quantile(0.25), 3)
            q3 = np.around(group_data[col].quantile(0.75), 3)
            num_summary[exercise_group][col] = f"{median} [{q1}, {q3}]"

    # Categorical variables
    for exercise_group, group_data in grouped:
        cat_summary[exercise_group] = {}
        for col in cat_vars:
            counts = group_data[col].value_counts()
            percentages = (counts / counts.sum()) * 100
            cat_summary[exercise_group][col] = [
                f"{category}: {count} ({percent:.2f}%)" 
                for category, count, percent in zip(counts.index, counts, percentages)
            ]

    # Return as DataFrames
    num_summary_df = pd.DataFrame(num_summary).T
    cat_summary_df = {
        group: pd.DataFrame.from_dict(cat_summary[group], orient='index').T 
        for group in cat_summary
    }

    return num_summary_df, cat_summary_df


In [ ]:
# Overall
overall_num, overall_cat = summarize_by_exercise(df_cut_final, num_vars, cat_vars)

# HELM only
helm_num, helm_cat = summarize_by_exercise(df_cut_final[df_cut_final['ID'].str.startswith('helm')], num_vars, cat_vars)

# DEXIP only
dexip_num, dexip_cat = summarize_by_exercise(df_cut_final[df_cut_final['ID'].str.startswith('dexip')], num_vars, cat_vars)


In [ ]:
dexip_num.T

,0,1
age,"13.0 [12.0, 14.0]","13.0 [12.0, 14.0]"
bmi,"20.596 [18.575, 23.565]","20.596 [18.575, 23.565]"
hba1c,"56.284 [49.454, 66.12]","56.284 [49.454, 66.12]"
years_since_diagnosis,"10.0 [7.0, 11.0]","10.0 [7.0, 11.0]"
glc_change,"-0.39 [-1.5, 0.845]","-1.67 [-3.44, 0.352]"
duration,"20.0 [15.0, 30.0]","20.0 [15.0, 30.0]"
iob_kg,"0.015 [0.004, 0.039]","0.017 [0.003, 0.04]"
glc,"12.195 [11.0, 14.72]","12.28 [10.94, 14.845]"
start_roc,"0.0 [-0.03, 0.05]","0.003 [-0.033, 0.05]"
cv,"6.131 [3.733, 10.213]","6.103 [3.349, 10.327]"


In [ ]:
overall_cat

{0:             insulin_modality                                       race  \
 0  closed_loop: 808 (48.15%)                       WHITE: 1517 (90.41%)   
 1         pump: 579 (34.51%)                   NOT REPORTED: 41 (2.44%)   
 2          mdi: 291 (17.34%)                          ASIAN: 41 (2.44%)   
 3                       None                       MULTIPLE: 39 (2.32%)   
 4                       None         BLACK/AFRICAN AMERICAN: 31 (1.85%)   
 5                       None  AMERICAN INDIAN/ALASKAN NATIVE: 6 (0.36%)   
 6                       None                         UNKNOWN: 3 (0.18%)   
 
                      sex                   hypo      form_of_exercise  \
 0  female: 1210 (72.11%)  False: 1678 (100.00%)  none: 1678 (100.00%)   
 1     male: 468 (27.89%)                   None                  None   
 2                   None                   None                  None   
 3                   None                   None                  None   
 4               

In [ ]:
overall_num_ov = summarize_by_exercise(df_demo_glyc, ov_num_vars, ov_cat_vars)
helm_num_ov = summarize_by_exercise(df_demo_glyc[df_demo_glyc['ID'].str.startswith('helm')], ov_num_vars, ov_cat_vars)
dexip_num_ov = summarize_by_exercise(df_demo_glyc[df_demo_glyc['ID'].str.startswith('dexip')], ov_num_vars, ov_cat_vars)

In [ ]:
overall_num_ov

(                 age                      bmi                    hba1c  \
 0  25.0 [16.0, 36.0]  23.804 [21.445, 26.573]  51.913 [46.448, 56.284]   
 1  28.0 [16.0, 40.0]  24.208 [22.045, 26.623]  49.727 [44.262, 56.284]   
 
   years_since_diagnosis Average glucose (mmol/L)              eA1c (%)  \
 0      12.0 [9.0, 18.5]     8.462 [7.677, 9.483]  6.951 [6.457, 7.593]   
 1      12.0 [9.0, 21.0]     8.289 [7.616, 9.541]   6.842 [6.419, 7.63]   
 
             SD (mmol/L)                   CV (%)        AUC (mmol h/L)  \
 0  2.884 [2.473, 3.416]  34.074 [30.449, 37.177]  7.701 [6.999, 8.628]   
 1   2.85 [2.418, 3.437]  34.072 [30.065, 37.692]   7.513 [6.916, 8.64]   
 
                    LBGI                  HBGI         MAGE (mmol/L)  \
 0  0.601 [0.276, 0.931]  5.661 [3.629, 8.645]   6.931 [6.051, 8.17]   
 1   0.648 [0.35, 1.018]  5.259 [3.403, 8.745]  6.946 [5.829, 8.269]   
 
            TIR normal (%)            tir_hypo               tir_hyper  
 0  71.65 [58.875, 80.125]  

### Start again

In [55]:
def summarize_participant_characteristics(df, num_vars, cat_vars):
    # Numerical variables
    num_summary = {}
    for col in num_vars:
        median = np.around(df[col].median(), 2)
        q1 = np.around(df[col].quantile(0.25), 2)
        q3 = np.around(df[col].quantile(0.75), 2)
        num_summary[col] = f"{median} [{q1}, {q3}]"
    num_df = pd.DataFrame(num_summary, index=["Summary"]).T

    # Categorical variables
    cat_summary = {}
    for col in cat_vars:
        counts = df[col].value_counts()
        percentages = (counts / counts.sum()) * 100
        cat_summary[col] = [
            f"{cat}: {cnt} ({pct:.1f}%)"
            for cat, cnt, pct in zip(counts.index, counts, percentages)
        ]
    cat_df = pd.DataFrame.from_dict(cat_summary, orient='index').T

    return num_df, cat_df


In [56]:
df_demo_glyc.shape[0]

482

In [57]:
overall_num_ov, overall_cat_ov = summarize_participant_characteristics(df_demo_glyc, ov_num_vars, ov_cat_vars)
helm_num_ov, helm_cat_ov = summarize_participant_characteristics(df_demo_glyc[df_demo_glyc['ID'].str.startswith('helm')], ov_num_vars, ov_cat_vars)
dexip_num_ov, dexip_cat_ov = summarize_participant_characteristics(df_demo_glyc[df_demo_glyc['ID'].str.startswith('dexip')], ov_num_vars, ov_cat_vars)


In [58]:
def summarize_bout_characteristics(df, num_vars, cat_vars):
    def _summarize(group):
        num_summary = {}
        cat_summary = {}

        for col in num_vars:
            median = np.around(group[col].median(), 2)
            q1 = np.around(group[col].quantile(0.25), 2)
            q3 = np.around(group[col].quantile(0.75), 2)
            num_summary[col] = f"{median} [{q1}, {q3}]"

        for col in cat_vars:
            counts = group[col].value_counts()
            percentages = (counts / counts.sum()) * 100
            cat_summary[col] = [
                f"{cat}: {cnt} ({pct:.1f}%)"
                for cat, cnt, pct in zip(counts.index, counts, percentages)
            ]

        return pd.Series({'numerical': num_summary, 'categorical': cat_summary})

    results = {}
    for label, subdf in {
        'Overall - PA': df[df['exercise'] == 1],
        'Overall - No PA': df[df['exercise'] == 0],
        'HELM - PA': df[df['ID'].str.startswith('helm') & (df['exercise'] == 1)],
        'HELM - No PA': df[df['ID'].str.startswith('helm') & (df['exercise'] == 0)],
        'DEXIP - PA': df[df['ID'].str.startswith('dexip') & (df['exercise'] == 1)],
        'DEXIP - No PA': df[df['ID'].str.startswith('dexip') & (df['exercise'] == 0)],
    }.items():
        results[label] = _summarize(subdf)

    return results


In [59]:
bout_summaries = summarize_bout_characteristics(df_cut_final, num_vars, cat_vars)


In [60]:
participant_num_table = pd.concat(
    [overall_num_ov, helm_num_ov, dexip_num_ov], 
    axis=1, 
    keys=["Overall", "HELM", "DEXIP"]
)


In [61]:
participant_cat_table = pd.concat(
    [overall_cat_ov, helm_cat_ov, dexip_cat_ov], 
    axis=0, 
    keys=["Overall", "HELM", "DEXIP"]
)


In [62]:
bout_num_table = pd.DataFrame({k: pd.Series(v["numerical"]) for k, v in bout_summaries.items()})


In [63]:
bout_cat_table = pd.DataFrame({k: pd.Series(v["categorical"]) for k, v in bout_summaries.items()})


In [64]:
participant_cat_table.T.to_csv('../../data/tidy/participant_cat_table.csv')
participant_num_table.to_csv('../../data/tidy/participant_num_table.csv')
bout_cat_table.to_csv('../../data/tidy/bout_cat_table.csv')
bout_num_table.to_csv('../../data/tidy/bout_num_table.csv')

In [80]:
bout_cat_table.reset_index(inplace=True)

In [98]:
bout_cat_table[bout_cat_table['index']=='intensity']['DEXIP - PA'].values

array([list(['0.0: 189 (74.7%)', '1.0: 51 (20.2%)', '2.0: 13 (5.1%)'])],
      dtype=object)

In [91]:
bout_cat_table

,index,Overall - PA,Overall - No PA,HELM - PA,HELM - No PA,DEXIP - PA,DEXIP - No PA
0,hypo,[False: 1546 (100.0%)],[False: 1546 (100.0%)],[False: 1293 (100.0%)],[False: 1293 (100.0%)],[False: 253 (100.0%)],[False: 253 (100.0%)]
1,form_of_exercise,"[aer: 875 (56.6%), mix: 537 (34.7%), ana: 134 ...",[none: 1546 (100.0%)],"[aer: 676 (52.3%), mix: 494 (38.2%), ana: 123 ...",[none: 1293 (100.0%)],"[aer: 199 (78.7%), mix: 43 (17.0%), ana: 11 (4...",[none: 253 (100.0%)]
2,time_of_day,"[afternoon: 700 (45.3%), evening: 456 (29.5%),...","[evening: 619 (40.0%), morning: 467 (30.2%), a...","[afternoon: 560 (43.3%), evening: 398 (30.8%),...","[evening: 518 (40.1%), afternoon: 402 (31.1%),...","[afternoon: 140 (55.3%), evening: 58 (22.9%), ...","[evening: 101 (39.9%), morning: 94 (37.2%), af..."
3,intensity,"[0.0: 768 (49.7%), 1.0: 653 (42.2%), 2.0: 125 ...",[-1.0: 1546 (100.0%)],"[1.0: 602 (46.6%), 0.0: 579 (44.8%), 2.0: 112 ...",[-1.0: 1293 (100.0%)],"[0.0: 189 (74.7%), 1.0: 51 (20.2%), 2.0: 13 (5...",[-1.0: 253 (100.0%)]


In [66]:
bout_num_table

,Overall - PA,Overall - No PA,HELM - PA,HELM - No PA,DEXIP - PA,DEXIP - No PA
glc_change,"-37.98 [-70.7, -7.92]","-5.04 [-21.96, 9.9]","-40.86 [-72.0, -10.98]","-5.04 [-20.88, 8.1]","-21.96 [-57.06, 6.12]","-5.04 [-25.02, 18.0]"
duration,"24.0 [20.0, 30.0]","24.0 [20.0, 30.0]","25.0 [20.0, 30.0]","25.0 [20.0, 30.0]","20.0 [15.0, 30.0]","20.0 [15.0, 30.0]"
iob_kg,"0.01 [0.0, 0.03]","0.02 [0.01, 0.03]","0.01 [0.0, 0.03]","0.02 [0.0, 0.03]","0.02 [0.0, 0.04]","0.02 [0.01, 0.05]"
glc,"210.06 [192.96, 235.08]","208.98 [192.06, 234.0]","208.98 [192.06, 232.02]","208.08 [192.06, 232.02]","214.92 [194.94, 252.0]","216.0 [194.94, 250.92]"
start_roc,"0.0 [-0.55, 0.7]","0.04 [-0.5, 0.75]","0.0 [-0.55, 0.7]","0.0 [-0.5, 0.7]","0.04 [-0.59, 0.8]","0.04 [-0.55, 0.76]"
cv,"5.86 [3.39, 9.55]","5.66 [3.48, 9.21]","5.93 [3.43, 9.88]","5.65 [3.41, 9.41]","5.58 [3.24, 8.65]","5.67 [3.73, 8.47]"


In [79]:
participant_cat_table

,level_0,level_1,insulin_modality,race,sex
0,Overall,0,closed_loop: 226 (46.9%),WHITE: 435 (90.2%),female: 329 (68.3%)
1,Overall,1,pump: 183 (38.0%),NOT REPORTED: 14 (2.9%),male: 153 (31.7%)
2,Overall,2,mdi: 73 (15.1%),MULTIPLE: 11 (2.3%),None
3,Overall,3,None,ASIAN: 10 (2.1%),None
4,Overall,4,None,BLACK/AFRICAN AMERICAN: 8 (1.7%),None
5,Overall,5,None,AMERICAN INDIAN/ALASKAN NATIVE: 2 (0.4%),None
6,Overall,6,None,UNKNOWN: 2 (0.4%),None
7,HELM,0,closed_loop: 160 (44.1%),WHITE: 328 (90.4%),female: 281 (77.4%)
8,HELM,1,pump: 148 (40.8%),NOT REPORTED: 10 (2.8%),male: 82 (22.6%)
9,HELM,2,mdi: 55 (15.2%),ASIAN: 9 (2.5%),None


In [77]:
participant_cat_table[participant_cat_table['level_0']=='DEXIP']['race'].values

array(['WHITE: 107 (89.9%)', 'MULTIPLE: 5 (4.2%)',
       'NOT REPORTED: 4 (3.4%)', 'ASIAN: 1 (0.8%)', 'UNKNOWN: 1 (0.8%)',
       'BLACK/AFRICAN AMERICAN: 1 (0.8%)'], dtype=object)

In [68]:
participant_num_table

,Overall,HELM,DEXIP
,Summary,Summary,Summary
age,"26.0 [18.0, 39.0]","32.0 [25.0, 44.5]","13.0 [12.0, 14.0]"
bmi,"24.06 [21.8, 26.61]","24.69 [23.05, 27.39]","20.8 [17.99, 23.96]"
hba1c,"50.82 [45.36, 56.28]","49.73 [44.26, 54.1]","54.1 [48.63, 61.75]"
years_since_diagnosis,"12.0 [9.0, 20.0]","15.0 [10.0, 23.0]","10.0 [7.0, 11.0]"
Average glucose (mmol/L),"151.06 [138.28, 171.91]","146.69 [136.11, 163.32]","169.87 [150.31, 186.61]"
eA1c (%),"6.91 [6.46, 7.64]","6.75 [6.38, 7.34]","7.56 [6.88, 8.15]"
SD (mmol/L),"51.8 [43.86, 61.94]","50.06 [42.68, 60.09]","58.95 [49.21, 68.51]"
CV (%),"34.12 [30.4, 37.42]","33.89 [29.94, 37.4]","34.24 [31.32, 37.57]"
AUC (mmol h/L),"136.56 [125.92, 156.84]","133.44 [122.6, 148.18]","154.3 [135.06, 169.06]"
